 # Week 11: Exercise 11.2
 File: DSC540_Paulovici_Exercise_11_2.py (.ipynb)<br>
 Name: Kevin Paulovici<br>
 Date: 2/23/2020<br>
 Course: DSC 540 Data Preparation (2203-1)<br>
 Assignment: Exercise 11.2

 Going back to Chapter 9 (Data Wrangling with Python), let’s practice joining numerous datasets – an activity you will likely run into frequently. Following the example in your text that starts on page 229 – 233 of Data Wrangling with Python, work through the example to bring two datasets together. Submit your code and output to the assignment link.

In [1]:
import xlrd
from xlrd.sheet import ctype_text
import agate

# import the UNICEF child labor data
cpi_workbook = xlrd.open_workbook('corruption_perception_index.xls')
cpi_sheet = cpi_workbook.sheets()[0]

for r in range(cpi_sheet.nrows):
    print(r, cpi_sheet.row_values(r))


0 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
1 ['Country Rank', 'Country / Territory', '', '', '', 'Country Rank', 'CPI 2013 Score', 'Surveys Used', 'Standard Error', '90% Confidence interval', '', 'Scores range', '', 'Data sources', '', '', '', '', '', '', '', '', '', '', '', '']
2 ['', '', 'WB Code', 'IFS Code', 'Region', '', '', '', '', 'Lower', 'Upper', 'MIN', 'MAX', 'AFDB', 'BF (SGI)', 'BF (BTI)', 'IMD', 'ICRG', 'WB', 'WEF', 'WJP', 'EIU', 'GI', 'PERC', 'TI', 'FH']
3 [1.0, 'Denmark', 'DNK', 128.0, 'EU', 1.0, 91.0, 7.0, 2.2, 87.0, 95.0, 83.0, 98.0, 0.0, 97.0, 0.0, 96.0, 98.0, 0.0, 87.0, 89.0, 88.0, 83.0, 0.0, 0.0, 0.0]
4 [1.0, 'New Zealand', 'NZL', 196.0, 'AP', 1.0, 91.0, 7.0, 2.3, 87.0, 95.0, 83.0, 98.0, 0.0, 97.0, 0.0, 89.0, 98.0, 0.0, 96.0, 86.0, 88.0, 83.0, 0.0, 0.0, 0.0]
5 [3.0, 'Finland', 'FIN', 172.0, 'EU', 3.0, 89.0, 7.0, 1.7, 86.0, 92.0, 83.0, 98.0, 0.0, 89.0, 0.0, 89.0, 98.0, 0.0, 90.0, 87.0, 88.0, 83.0, 0.0, 0.

In [2]:
# get titles
cpi_title_rows = zip(cpi_sheet.row_values(1), cpi_sheet.row_values(2))
cpi_titles = [t[0] + ' ' + t[1] for t in cpi_title_rows]
cpi_titles = [t.strip() for t in cpi_titles]

# get rows
cpi_rows = [cpi_sheet.row_values(r) for r in range (3, cpi_sheet.nrows)]

# data types
text_type = agate.Text()
number_type = agate.Number()
bool_type = agate.Boolean()
date_type = agate.Date()

# get types
def get_types(example_row):
    """ function determines the type of data"""
    types = []

    for v in example_row:
        value_type = ctype_text[v.ctype]
        if value_type == 'text':
            types.append(text_type)
        elif value_type == 'number':
            types.append(number_type)
        elif value_type == 'xldate':
            types.append(date_type)
        else:
            types.append(text_type)

    return types

cpi_types = get_types(cpi_sheet.row(3))


In [3]:
def get_table(new_arr, types, titles):
    """ function handles errors """
    try:
        table = agate.Table(new_arr, titles, types)
        return table
    except Exception as e:
        print(e)


In [4]:
# create a table of our data
cpi_table = get_table(cpi_rows, cpi_types, cpi_titles)


C:\Users\Kevin\Anaconda3\lib\site-packages\agate\utils.py:292: DuplicateColumnWarning: Column name "Country Rank" already exists in Table. Column will be renamed to "Country Rank_2".


In [5]:
# to reduce repetetive code, we can import the 'ranked' table from week 7
# This would be a modified version to remove prints and output
from DSC540_Paulovici_Exercise_7_2 import ranked

print(type(ranked))


C:\Users\Kevin\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
C:\Users\Kevin\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
C:\Users\Kevin\Anaconda3\lib\site-packages\agate\utils.py:276: UnnamedColumnWarning: Column 2 has no name. Using "c".


<Figure size 640x480 with 1 Axes>

<class 'agate.table.Table'>


In [6]:
# join our child labor data and our cpi_table
cpi_and_cl = cpi_table.join(ranked, 'Country / Territory', 'Countries and areas', inner=True)

cpi_and_cl.column_names

('Country Rank',
 'Country / Territory',
 'WB Code',
 'IFS Code',
 'Region',
 'Country Rank_2',
 'CPI 2013 Score',
 'Surveys Used',
 'Standard Error',
 '90% Confidence interval Lower',
 'Upper',
 'Scores range MIN',
 'MAX',
 'Data sources AFDB',
 'BF (SGI)',
 'BF (BTI)',
 'IMD',
 'ICRG',
 'WB',
 'WEF',
 'WJP',
 'EIU',
 'GI',
 'PERC',
 'TI',
 'FH',
 'Total (%)',
 'c',
 'Sex (%)Male',
 'Female',
 'Place of residence (%)Urban',
 'Rural',
 'Household wealth quintile (%)Poorest',
 'Second',
 'Middle',
 'Fourth',
 'Richest',
 'Reference Year',
 'Data Source',
 'Children not working (%)',
 'Total Child Labor Rank')

In [7]:
# print out the first few rows of data
for r in cpi_and_cl.order_by('CPI 2013 Score').limit(10).rows:
    print('{}: {} - {}%'.format(r['Country / Territory'], r['CPI 2013 Score'],
    r['Total (%)']))


Afghanistan: 8.0 - 10.3%
Somalia: 8.0 - 49.0%
Iraq: 16.0 - 4.7%
Yemen: 18.0 - 22.7%
Chad: 19.0 - 26.1%
Equatorial Guinea: 19.0 - 27.8%
Guinea-Bissau: 19.0 - 38.0%
Haiti: 19.0 - 24.4%
Cambodia: 20.0 - 18.3%
Burundi: 21.0 - 26.3%
